# Read in test.tsv from model

In [14]:
import pandas as pd
import re
import numpy as np
path = 'C:/Users/Leste/Desktop/'
data=pd.read_csv(path+'test.tsv',sep=' ',header=None)
data.columns=['text','ori_lab','new_lab']
data['pre'] = data['ori_lab'].apply(lambda x: re.sub('-.*','',x))
data['class_'] = data['ori_lab'].apply(lambda x: re.sub('.*-','',x))
data['class_n'] = data['new_lab'].apply(lambda x: re.sub('.*-','',x))
nd = data[data['class_']!='O'].reset_index(drop=True)
B_index = nd[nd['pre']=='B'].index
# 0: TP, 1: FP, 2: FN
pre_conf = np.zeros((len(B_index), 3))

In [17]:
for i in range(len(B_index)):
    if i != len(B_index)-1:
        if B_index[i] + 1 == B_index[i+1]:
            true_lab = nd.loc[B_index[i],'ori_lab']
            pred_lab = nd.loc[B_index[i],'new_lab']
            if true_lab == pred_lab:
                pre_conf[i,0] = 1
            elif (true_lab != pred_lab) & (pred_lab != 'O'):
                pre_conf[i,1] = 1
            else:
                pre_conf[i,2] = 1
        
        else:
            true_lab = nd.loc[B_index[i]:(B_index[i+1]-1),'ori_lab']
            pred_lab = nd.loc[B_index[i]:(B_index[i+1]-1),'new_lab']
            if sum(true_lab == pred_lab) == len(true_lab):
                pre_conf[i,0] = 1
            elif sum([x == 'O' for x in pred_lab]) == len(true_lab):
                pre_conf[i,2] = 1
            else:
                pre_conf[i,1] = 1 
                pre_conf[i,2] = 1
    else:
        true_lab = nd.loc[B_index[i]:,'ori_lab']
        pred_lab = nd.loc[B_index[i]:,'new_lab']
        if sum(true_lab == pred_lab) == len(true_lab):
            pre_conf[i,0] = 1
        elif sum([x == 'O' for x in pred_lab]) == len(true_lab):
            pre_conf[i,2] = 1
        else:
            pre_conf[i,1] = 1
            pre_conf[i,2] = 1


False

In [23]:
conf = pd.DataFrame(pre_conf, columns=['TP','FP','FN'])
conf['class_'] = nd.loc[B_index,'class_'].reset_index(drop=True)
conf.groupby('class_').sum()

,TP,FP,FN
class_,,,
CAD,4.0,35.0,719.0
DIABETES,0.0,7.0,737.0
FAMILY_HIST,0.0,0.0,24.0
HYPERLIPIDEMIA,0.0,41.0,295.0
HYPERTENSION,91.0,6.0,671.0
MEDICATION,1491.0,383.0,1125.0
OBESE,0.0,0.0,129.0
SMOKER,0.0,12.0,561.0


In [5]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
y_true = [data.ori_lab.values.tolist()]
y_pred = [data.new_lab.values.tolist()]
f1_score(y_true, y_pred)
a = classification_report(y_true, y_pred)

c:\Users\Leste\anaconda3\envs\dataweek2\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
print(a)

                precision    recall  f1-score   support

           CAD       0.08      0.01      0.01       731
      DIABETES       0.00      0.00      0.00       738
   FAMILY_HIST       0.00      0.00      0.00        24
HYPERLIPIDEMIA       0.00      0.00      0.00       336
  HYPERTENSION       0.61      0.12      0.20       763
    MEDICATION       0.58      0.57      0.57      2616
         OBESE       0.00      0.00      0.00       129
        SMOKER       0.00      0.00      0.00       562

     micro avg       0.57      0.27      0.37      5899
     macro avg       0.16      0.09      0.10      5899
  weighted avg       0.34      0.27      0.28      5899



In [16]:
data[(data.class_ != "O")].groupby('class_').apply(lambda x: sum(x.new_lab == x.ori_lab))
#data[(data.class_ != "O")].groupby('class_').apply(lambda x: sum((x.new_lab != x.ori_lab) & (x.new_lab !="O")))

class_
CAD                 25
DIABETES             0
FAMILY_HIST          0
HYPERLIPIDEMIA       0
HYPERTENSION        92
MEDICATION        1824
OBESE                0
SMOKER               0
dtype: int64